In [ ]:
!pip install keras-tuner -q

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import keras_tuner as kt
from kaggle_secrets import UserSecretsClient
import os

In [ ]:
secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")

In [ ]:
!kaggle kernels output ankitk2001/btp-cnn -p /kaggle/working

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
xDim = 256
yDim = 256

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/train',
    seed=123,
    image_size=(xDim,yDim),
    batch_size=32
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid',
    seed=123,
    image_size=(xDim,yDim),
    batch_size=32
)

In [ ]:
numClass = len(train_ds.class_names)
print(numClass,train_ds.class_names)

In [ ]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(xDim, yDim, 3)))
  model.add(tf.keras.layers.Rescaling(1.0/255))

  activations = 'relu'
  for i in range(hp.Int("num_conv_layers", 2, 7)):
    model.add(
        tf.keras.layers.Conv2D(
            filters=hp.Int(f"filter_{i}", min_value=32, max_value=128, step=32),
            kernel_size = (3,3),
            activation=activations,
        )
    )
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

  model.add(tf.keras.layers.Flatten())

  for i in range(hp.Int('num_dense_layers',1,3)):
    model.add(
        tf.keras.layers.Dense(
            units = hp.Int(f"unit_{i}", min_value=64, max_value=256, step=64),
            activation = activations,
            kernel_regularizer=tf.keras.regularizers.L2(l2=hp.Float(f"lambda_{i}",min_value=0.001,max_value=0.1,step=10,sampling='log'))
        )
    )

  model.add(tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=hp.Float("lambda",min_value=0.001,max_value=0.1,step=10,sampling='log'))))
  model.add(tf.keras.layers.Dense(1,activation='linear'))

  model.compile(optimizer=hp.Choice('optimizer',['adam','rmsprop']),
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

  return model


In [ ]:
tuner1 = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='models',
                     project_name='BTP_CNN')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner1.search(train_ds,validation_data=val_ds, epochs=50,callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner1.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
model = tuner1.hypermodel.build(best_hps)

In [ ]:
model.save('/kaggle/working/untrained_model_btp')

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tf.keras.models.load_model('/kaggle/working/untrained_model_btp')

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = '/kaggle/working/finalModel/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(train_ds, epochs=50, validation_data=val_ds,callbacks=[model_checkpoint_callback,stop_early])

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
model.save('/kaggle/working/trained_btp_cnn')

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/trained_btp_cnn')

In [ ]:
tf.keras.utils.plot_model(model, to_file='/kaggle/working/model.png', show_shapes=True)

In [ ]:
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.plot(history.history['val_accuracy'])
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.show()

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test',
    image_size=(xDim,yDim),
)

In [ ]:
loss,acc = model.evaluate(test_ds,verbose=2)
acc*100

In [ ]:
imageCounter = 1;
for images,labels in test_ds:
    batchPredictions = model.predict(images).reshape((32));
    roundedVals = tf.cast(tf.math.round(tf.math.sigmoid((batchPredictions))),'int32')
    correctAns = 0
    for idx in range(len(roundedVals)):
#         if roundedVals[idx]==labels[idx]:
#             correctAns+=1
        plt.subplot(4,4,imageCounter)
        plt.imshow(images[idx]/255.0)
        plt.axis('off')
        plt.title(f'{test_ds.class_names[labels[idx]]}/{test_ds.class_names[roundedVals[idx]]}')
        imageCounter = imageCounter + 1
        if(imageCounter==17):
            break
    print(correctAns)
    break
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)    

plt.show()

In [ ]:
x_train = []
y_train = []
for i,l in test_ds:
  val = model.predict(i);
  for j in range(len(val)):
    x_train.append(val[j][0])
    y_train.append(l[j])

for i in range(len(y_train)):
  y_train[i] = y_train[i].numpy()

x_np_train =pd.DataFrame(x_train)
y_np_train = pd.DataFrame(y_train)
print(x_np_train.shape,y_np_train.shape)

In [ ]:
x_np_train

In [ ]:
yhat = tf.cast(tf.math.round(tf.math.sigmoid((x_np_train))),'int32')

In [ ]:
yhatnp = yhat.numpy().reshape((20000))

In [ ]:
yrealnp = np.array(y_train).reshape((20000))

In [ ]:
cm = confusion_matrix(yrealnp,yhatnp)

In [ ]:
categories = ['Fake','Real']
sns.heatmap(cm,xticklabels=['Fake','Real'],yticklabels = ['Fake','Real'],annot=True,fmt='g',cmap="Blues")
plt.title('DeepFake CNN (BTP) Confusion Matrix')
plt.ylabel('Actual',fontsize=13)
plt.xlabel('Prediction',fontsize=13)
plt.show()

In [ ]:
print(classification_report(yrealnp,yhatnp))